# 程序员的数学基础

## 二进制

二进制左移一位就是乘以2

In [8]:
a = 1
b = a << 1
assert(b == a*2)
c = a << 3
assert(c == a*(2**3))

二进制右移一位就是除以2求整数商

In [20]:
a = 53001
b = a >> 1
assert(b == (a // 2))
c = a >> 4
assert(c == (a // (2**4)))

1. 位与：两个位必须全部为1结果才为1
2. 位或：两个位只要有一个为1，结果就为1
3. 异或：两个位相同，结果为0；两个位不同，结果为1

## 余数

同余定理：<br>
两个整数a,b,如果他们除以同一个数m后余数相等，我们就可以说a和b对于模m同余。<br>
同余定理其实就是用来分类的。

In [22]:
a = 5
print("5 除以 2 余 ", 5%2)

5 除以 2 余  1


取余操作也可以当作hash来用

## 迭代法
迭代法就是不断用旧的变量计算出新的变量的值<br>
它很适合用循环语句来实现<br>
二分法就是迭代法的一个经典例子<br>
下面的例子就是使用二分法来计算平方根，这里面不断迭代的变量就是mid<br>

In [57]:
import math

def it_sqrt(num, precision = 10**-10, max_it_count=10**6):
    if num < 0:
        raise ValueError("num should > 0")
    small = 0
    big = num
    for i in range (max_it_count):
        mid = (big + small) / 2
        loss = mid**2 - num
        #print("i=%d mid=%f loss = %f" %(i, mid, loss))
        if abs(loss) < precision:
            print("get res i=", i)
            return (mid, loss)
        elif loss > 0: # mid is too big, pick small ~ mid
            big = mid
        else: # mid is too small, pick mid ~ big
            small = mid
    return(mid, loss)
    raise RuntimeError("can not get result in %d loss=%f" % (max_it_count, loss))
num = 5000000000000
print(math.sqrt(num))
res, loss = it_sqrt(num)
print("result=%f loss=%f"% (res,loss))

2236067.9774997896
result=2236067.977500 loss=-0.000977


## 递归法
下面的归并排序使用了分治的思想，采用了递归的实现方法。<br>
1. 将要排序的数组每次都分成左右两半，一直分下去，直到只剩下一个元素，这个元素就是天然排序好的数组。
2. 将排序好的左右两个数组归并成一个排序好的数组。

In [4]:
import random
def merge(a, b):
    # a and b is sorted
    c = []
    while( len(a) != 0 and len(b) != 0):
        if a[0] < b[0]:
            c.append(a[0])
            a = a[1:]
        elif a[0] > b[0]:
            c.append(b[0])
            b = b[1:]
        else:
            c.append(a[0])
            c.append(b[0])
            a = a[1:]
            b = b[1:]
    if len(a) != 0:
        c += a
    if len(b) != 0:
        c += b
    return c


def merge_sort(l):
    if len(l) == 0:
        return []
    if len(l) == 1:
        return l
    mid = len(l)//2
    left = l[0:mid]
    right = l[mid:]
    left = merge_sort(left)
    right = merge_sort(right)
    merged = merge(left, right)
    return merged

l  = [random.randint(0,  20) for _ in range(10)]
print(l)
sorted_l = merge_sort(l)
print(sorted_l)

[1, 6, 8, 3, 4, 15, 6, 7, 19, 9]
[1, 3, 4, 6, 6, 7, 8, 9, 15, 19]


### 理解分治的思想:MapReduce
MapReduce 大体上分为input,split,map,shuffle,reduce,output几个步骤

#### 第一步是获取input数据

In [6]:
#1. get input
def get_input_data(file_path):
    with open(file_path) as f:
        content = f.read()
    return content
data = get_input_data("res/word_count.txt")
print("-----input-----")
print(data)

-----input-----
Hello Java
Hello C  
Hello Java
Hello C++
Hello Go
Hello Python


#### 第二步是将原始数据分割，分给多个mapper

In [7]:
def do_split(input):
    lines = input.split('\n')
    cont = dict((k, v.strip()) for k, v in enumerate(lines))
    return cont
data = do_split(data)
print("-------split-------")
print(data)

-------split-------
{0: 'Hello Java', 1: 'Hello C', 2: 'Hello Java', 3: 'Hello C++', 4: 'Hello Go', 5: 'Hello Python'}


#### 第三步里各个mapper并行地将其获取的数据根据相应的业务逻辑映射成key-value

In [8]:
def mapper_map(splited_item):
    words = splited_item.split(' ')
    mapped = [(v,1) for k, v in enumerate(words)]
    return mapped
#3. 为了实现简单，我们采用for循环来进行map,实际是可以有n个mapper来并行的做map的
mapped_list = []
for k, v in data.items():
    mapped = mapper_map(v)
    mapped_list += mapped
print("------mapped-----")
print(mapped_list)

------mapped-----
[('Hello', 1), ('Java', 1), ('Hello', 1), ('C', 1), ('Hello', 1), ('Java', 1), ('Hello', 1), ('C++', 1), ('Hello', 1), ('Go', 1), ('Hello', 1), ('Python', 1)]


#### 第四步是根据上面映射的key来作分类，分配到相应的reducer上去

In [9]:
#4. shuffle,将相同的(或相关的，具体根据业务将数据按照key来进行分组)key的data分发到相应的机器上去
hello_machine = []
java_machine = []
cxx_machine = []
go_machine = []
python_machine = []
for (k,v) in mapped_list:
    if k == "Hello":
        hello_machine.append((k,v))
    if k == "Java":
        java_machine.append((k,v))
    if k == "C++":
        cxx_machine.append((k,v))
    if k == "Go":
        go_machine.append((k,v))
    if k == "Python":
        python_machine.append((k,v))
        
print("--------after shuffle--------")
print(hello_machine)
print(java_machine)
print(cxx_machine)
print(go_machine)
print(python_machine)

--------after shuffle--------
[('Hello', 1), ('Hello', 1), ('Hello', 1), ('Hello', 1), ('Hello', 1), ('Hello', 1)]
[('Java', 1), ('Java', 1)]
[('C++', 1)]
[('Go', 1)]
[('Python', 1)]


#### 第五步是各个reducer并发的作reduce

In [10]:
def do_reduce(datas):
    
    if len(datas) == 0:
        return
    key = datas[0][0]
    return (key, len(datas))

hello_output = do_reduce(hello_machine)
java_output = do_reduce(java_machine)
cxx_output = do_reduce(cxx_machine)
go_output = do_reduce(go_machine)
python_output = do_reduce(python_machine)

#### 最后一步是收集汇总各个reducer产生的输出

In [11]:
output = []
output.append(hello_output)
output.append(java_output)
output.append(cxx_output)
output.append(go_output)
output.append(python_output)
print(output)

[('Hello', 6), ('Java', 2), ('C++', 1), ('Go', 1), ('Python', 1)]
